# Классификация

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import TomekLinks

from sklearn. model_selection import train_test_split
from sklearn. model_selection import RepeatedStratifiedKFold
from sklearn. model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn. discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

from typing import Dict
random.seed(1)
sns.set_theme(style="ticks")

from seaborn import load_dataset, pairplot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib.colors import ListedColormap
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# Первичная обработка данных

In [3]:
df = pd.read_csv('predict_sex_train_utf.csv', delimiter=',')
df_kag = pd.read_csv('predict_sex_test_no_answers_no_mv.csv', delimiter=',')
df.head()

,Id,Sex,НЕВЕРОТН 1 Невероятные приключения итальянцев в России,ТЕЛОХРАН 2 Телохранитель,КАВКАЗСК 3 Кавказская пленница или Новые приключения Шурика,ПТЫЙ_ЭЛЕ 4 Пятый элемент,ТЕРМИНАТ 5 Терминатор,КРЕПКИЙ 6 Крепкий орешек,ПОЛИЦ_АК 7 Полицейская академия,ЧУЖИЕ 8 Чужие,...,РУКА_ПЕР 191 Рука-переплетение пальцев рук - ведущая,РУКА_ИМИ 192 Рука-имитация аплодисментов - ведущая,РУКА_СКР 193 Рука-скрещивание рук на груди - ведущая,HОГА 194 Hога - ведущая,ГЛАЗ 195 Глаз - ведущая,РУКОЙ_ПИ 196 Рука которой пишут,ПЕРЕУЧИВ 197 Переучивали ли писать если левша,САМООПPЕ 198 Самоопpеделение пpавша-левша,БОК_ПРЕД 199 Бок на котором предпочитают спать,ГЛАЗ_У_В 200 Глаз который обычно шире открыт
0,1,2.0,5.0,6.0,8.0,11.7,8.7,8.0,8.0,5.0,...,1.0,2.0,2.0,2.0,2.0,1.0,4.0,1.0,2.0,2.0
1,2,1.0,7.0,7.0,7.0,10.0,3.0,9.0,6.2,12.0,...,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,3.0
2,3,1.0,12.0,7.0,6.0,10.0,7.0,8.0,7.0,6.0,...,2.0,1.0,2.0,1.0,2.0,1.0,4.0,2.0,3.0,2.0
3,4,1.0,7.0,8.0,12.0,7.0,12.0,11.0,9.0,12.0,...,2.0,2.0,1.0,2.0,2.0,1.0,NaN,1.0,3.0,1.0
4,5,1.0,5.0,9.0,2.0,11.0,8.0,6.0,2.0,3.0,...,1.0,2.0,2.0,2.0,2.0,1.0,4.0,1.0,1.0,2.0


Из обучающей выборки удалим строки с NA в целевой переменной.

In [4]:
df = df.dropna(subset = ['Sex'])
#df = df.dropna(axis = 1, thresh=2*len(df)/3)
df

,Id,Sex,НЕВЕРОТН 1 Невероятные приключения итальянцев в России,ТЕЛОХРАН 2 Телохранитель,КАВКАЗСК 3 Кавказская пленница или Новые приключения Шурика,ПТЫЙ_ЭЛЕ 4 Пятый элемент,ТЕРМИНАТ 5 Терминатор,КРЕПКИЙ 6 Крепкий орешек,ПОЛИЦ_АК 7 Полицейская академия,ЧУЖИЕ 8 Чужие,...,РУКА_ПЕР 191 Рука-переплетение пальцев рук - ведущая,РУКА_ИМИ 192 Рука-имитация аплодисментов - ведущая,РУКА_СКР 193 Рука-скрещивание рук на груди - ведущая,HОГА 194 Hога - ведущая,ГЛАЗ 195 Глаз - ведущая,РУКОЙ_ПИ 196 Рука которой пишут,ПЕРЕУЧИВ 197 Переучивали ли писать если левша,САМООПPЕ 198 Самоопpеделение пpавша-левша,БОК_ПРЕД 199 Бок на котором предпочитают спать,ГЛАЗ_У_В 200 Глаз который обычно шире открыт
0,1,2.0,5.0,6.0,8.0,11.7,8.7,8.0,8.0,5.0,...,1.0,2.0,2.0,2.0,2.0,1.0,4.0,1.0,2.0,2.0
1,2,1.0,7.0,7.0,7.0,10.0,3.0,9.0,6.2,12.0,...,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,3.0
2,3,1.0,12.0,7.0,6.0,10.0,7.0,8.0,7.0,6.0,...,2.0,1.0,2.0,1.0,2.0,1.0,4.0,2.0,3.0,2.0
3,4,1.0,7.0,8.0,12.0,7.0,12.0,11.0,9.0,12.0,...,2.0,2.0,1.0,2.0,2.0,1.0,NaN,1.0,3.0,1.0
4,5,1.0,5.0,9.0,2.0,11.0,8.0,6.0,2.0,3.0,...,1.0,2.0,2.0,2.0,2.0,1.0,4.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,995,2.0,6.0,6.5,6.5,7.0,7.0,6.0,6.5,5.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0
995,996,2.0,7.0,5.0,3.0,6.0,1.0,4.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,998,1.0,7.0,7.0,7.5,10.0,11.0,9.0,9.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,999,1.0,7.0,10.0,7.0,10.0,7.0,8.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_kag = df_kag.loc[:, df.columns.drop('Sex')]
df = df.fillna(df.median())
df_kag_NA = df_kag.fillna(df_kag.median())

In [6]:
df1_train, df1_test = train_test_split(df, test_size=0.25)

Проверим соотношение классов:

In [7]:
df1_train['Sex'].value_counts()

1.0    341
2.0    326
Name: Sex, dtype: int64

In [8]:
df1_test['Sex'].value_counts()

2.0    123
1.0    100
Name: Sex, dtype: int64

In [9]:
Xtrain = df1_train.drop(['Sex', 'Id'], axis=1)
ytrain =  df1_train[['Sex']].astype(int)
Xtest = df1_test.drop(['Sex', 'Id'], axis=1)
ytest = df1_test['Sex'].astype(int)

In [10]:
X_train, Y_train =  Xtrain, ytrain
X_test, Y_test = Xtest, ytest

## Дискриминантный анализ

Предположим, что ковариационные матрицы двух групп равны и проведем линейный дискриминантный анализ.

In [11]:
lda = LinearDiscriminantAnalysis()
lda.fit(Xtrain, ytrain)

lda_pr = lda.predict(Xtest)
print(classification_report(ytest, lda_pr))

print('train accuracy:', lda.score(Xtrain, ytrain))
print('test accuracy:', lda.score(Xtest, ytest))

              precision    recall  f1-score   support

           1       0.70      0.76      0.73       100
           2       0.79      0.74      0.76       123

    accuracy                           0.75       223
   macro avg       0.75      0.75      0.75       223
weighted avg       0.75      0.75      0.75       223

train accuracy: 0.881559220389805
test accuracy: 0.7488789237668162


In [12]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(Xtrain, ytrain)

qda_pr = qda.predict(Xtest)
print(classification_report(ytest, qda_pr))

print('train accuracy:', qda.score(Xtrain, ytrain))
print('test accuracy:', qda.score(Xtest, ytest))

              precision    recall  f1-score   support

           1       0.52      0.73      0.61       100
           2       0.67      0.46      0.54       123

    accuracy                           0.58       223
   macro avg       0.60      0.59      0.58       223
weighted avg       0.61      0.58      0.57       223

train accuracy: 0.992503748125937
test accuracy: 0.57847533632287


Качество классификации на тестовом наборе немного уменьшилось в сравнении с lda. 
При этом стоит помнить, что для этого нам понадобилось оценить намного больше параметров.

Перейдем к модели наивного байесовского классификатора:

In [13]:
gnb = GaussianNB()
gnb.fit(Xtrain, ytrain)

gnb_pr = gnb.predict(Xtest)
print(classification_report(ytest, gnb_pr))

print('train accuracy:', gnb.score(Xtrain, ytrain))
print('test accuracy:', gnb.score(Xtest, ytest))

              precision    recall  f1-score   support

           1       0.66      0.58      0.62       100
           2       0.69      0.76      0.72       123

    accuracy                           0.68       223
   macro avg       0.67      0.67      0.67       223
weighted avg       0.68      0.68      0.67       223

train accuracy: 0.7256371814092953
test accuracy: 0.6771300448430493


## Метод опорных векторов

In [14]:
C = 1  # параметр регуляризации

Для начала попробуем классический SVС (с линейной разделяющей плоксотью)

In [15]:
model_linear = svm.SVC(kernel='linear', C=C)
model_linear.fit(X_train, Y_train)

y_predict = model_linear.predict(X_test)

print(classification_report(Y_test, y_predict))

print('train accuracy:', model_linear.score(X_train, Y_train))
print('test accuracy:',model_linear.score(X_test, Y_test))

              precision    recall  f1-score   support

           1       0.74      0.81      0.77       100
           2       0.83      0.76      0.80       123

    accuracy                           0.78       223
   macro avg       0.78      0.79      0.78       223
weighted avg       0.79      0.78      0.79       223

train accuracy: 0.9175412293853074
test accuracy: 0.7847533632286996


В качестве одного из параметров мы можем выбрать ядро.
В реализации sklearn помимо линейного ядра можно выбрать радиальное, полиномиальное, сигмовидное ядро или написать свою функцию.
Поскольку ярко выраженной зависимости в данных не наблюдается, попробуем нелинейные ядра.

In [16]:
model_rbf = svm.SVC(kernel='rbf', C=C)
model_rbf.fit(X_train, Y_train)

y_predict = model_rbf.predict(X_test)

print(classification_report(Y_test, y_predict))

print('train accuracy:', model_rbf.score(X_train, Y_train))
print('test accuracy:',model_rbf.score(X_test, Y_test))

              precision    recall  f1-score   support

           1       0.58      0.80      0.67       100
           2       0.76      0.53      0.62       123

    accuracy                           0.65       223
   macro avg       0.67      0.66      0.65       223
weighted avg       0.68      0.65      0.65       223

train accuracy: 0.6746626686656672
test accuracy: 0.6502242152466368


In [17]:
model_poly4 = svm.SVC(kernel='poly', degree=4, C=C)
model_poly4.fit(X_train, Y_train)

y_predict = model_poly4.predict(X_test)

print(classification_report(Y_test, y_predict))

print('train accuracy:', model_poly4.score(X_train, Y_train))
print('test accuracy:',model_poly4.score(X_test, Y_test))

              precision    recall  f1-score   support

           1       0.71      0.67      0.69       100
           2       0.74      0.77      0.76       123

    accuracy                           0.73       223
   macro avg       0.72      0.72      0.72       223
weighted avg       0.73      0.73      0.73       223

train accuracy: 0.7691154422788605
test accuracy: 0.726457399103139


In [18]:
model_poly7 = svm.SVC(kernel='poly', degree=7, C=C)
model_poly7.fit(X_train, Y_train)

y_predict = model_poly7.predict(X_test)

print(classification_report(Y_test, y_predict))

print('train accuracy:', model_poly7.score(X_train, Y_train))
print('test accuracy:',model_poly7.score(X_test, Y_test))

              precision    recall  f1-score   support

           1       0.73      0.70      0.71       100
           2       0.76      0.79      0.78       123

    accuracy                           0.75       223
   macro avg       0.75      0.74      0.75       223
weighted avg       0.75      0.75      0.75       223

train accuracy: 0.8035982008995503
test accuracy: 0.7488789237668162


In [19]:
model_poly13 = svm.SVC(kernel='poly', degree=13, C=C)
model_poly13.fit(X_train, Y_train)

y_predict = model_poly13.predict(X_test)

print(classification_report(Y_test, y_predict))

print('train accuracy:', model_poly13.score(X_train, Y_train))
print('test accuracy:',model_poly13.score(X_test, Y_test))

              precision    recall  f1-score   support

           1       0.73      0.72      0.72       100
           2       0.77      0.78      0.78       123

    accuracy                           0.75       223
   macro avg       0.75      0.75      0.75       223
weighted avg       0.75      0.75      0.75       223

train accuracy: 0.8455772113943029
test accuracy: 0.7533632286995515


In [20]:
model_sigm0 = svm.SVC(kernel='sigmoid', C=C, coef0 = 0)
model_sigm0.fit(X_train, Y_train)

y_predict = model_sigm0.predict(X_test)

print(classification_report(Y_test, y_predict))

print('train accuracy:', model_sigm0.score(X_train, Y_train))
print('test accuracy:',model_sigm0.score(X_test, Y_test))

              precision    recall  f1-score   support

           1       0.45      1.00      0.62       100
           2       0.00      0.00      0.00       123

    accuracy                           0.45       223
   macro avg       0.22      0.50      0.31       223
weighted avg       0.20      0.45      0.28       223

train accuracy: 0.5112443778110944
test accuracy: 0.4484304932735426


In [21]:
model_sigm1 = svm.SVC(kernel='sigmoid', C=C, coef0 = 1)
model_sigm1.fit(X_train, Y_train)

y_predict = model_sigm1.predict(X_test)

print(classification_report(Y_test, y_predict))

print('train accuracy:', model_sigm1.score(X_train, Y_train))
print('test accuracy:',model_sigm1.score(X_test, Y_test))

              precision    recall  f1-score   support

           1       0.45      1.00      0.62       100
           2       0.00      0.00      0.00       123

    accuracy                           0.45       223
   macro avg       0.22      0.50      0.31       223
weighted avg       0.20      0.45      0.28       223

train accuracy: 0.5112443778110944
test accuracy: 0.4484304932735426


In [22]:
model_sigm10 = svm.SVC(kernel='sigmoid', C=C, coef0 = 10)
model_sigm10.fit(X_train, Y_train)

y_predict = model_sigm10.predict(X_test)

print(classification_report(Y_test, y_predict))

print('train accuracy:', model_sigm10.score(X_train, Y_train))
print('test accuracy:',model_sigm10.score(X_test, Y_test))

              precision    recall  f1-score   support

           1       0.45      1.00      0.62       100
           2       0.00      0.00      0.00       123

    accuracy                           0.45       223
   macro avg       0.22      0.50      0.31       223
weighted avg       0.20      0.45      0.28       223

train accuracy: 0.5112443778110944
test accuracy: 0.4484304932735426


## Логистическая регрессия

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [24]:
df1_train.quality = df1_train.Sex.astype(int)

In [25]:
df1_test.quality = df1_test.Sex.astype(int)

Попробуем построить модель с разными методами оптимизации. Метод Ньютона-Рафсона:

In [26]:
model = LogisticRegression(random_state = 12345, solver = 'newton-cg')
model.fit(Xtrain, ytrain)
predicted = model.predict_proba(Xtest)[:, 1]
ypred = model.predict(Xtest)
result = roc_auc_score(ytest, predicted)
accuracy = model.score(Xtest, ytest)

print('AUC-ROC:', result)
print('accuracy:', accuracy)

AUC-ROC: 0.8258536585365854
accuracy: 0.7623318385650224


Метод стохастического градиента:

In [27]:
model = LogisticRegression(random_state = 12345, solver = 'sag')
model.fit(Xtrain, ytrain)
predicted = model.predict_proba(Xtest)[:, 1]
ypred = model.predict(Xtest)
result = roc_auc_score(ytest, predicted)
accuracy = model.score(Xtest, ytest)

print('AUC-ROC:', result)
print('accuracy:', accuracy)

AUC-ROC: 0.8456910569105691
accuracy: 0.7668161434977578


Coordinate Descent (CD) algorithm:

In [28]:
model = LogisticRegression(random_state = 12345, solver = 'liblinear')
model.fit(Xtrain, ytrain)
predicted = model.predict_proba(Xtest)[:, 1]
ypred = model.predict(Xtest)
result = roc_auc_score(ytest, predicted)
accuracy = model.score(Xtest, ytest)

print('AUC-ROC:', result)
print('accuracy:', accuracy)

AUC-ROC: 0.8258536585365853
accuracy: 0.7533632286995515


По умолчанию --- метод Limited-memory Broyden–Fletcher–Goldfarb–Shanno Algorithm (solver = 'lbfgs'). Аналог метода Ньютона, но гессиан вычисляется приближенно. 

In [29]:
model = LogisticRegression(random_state = 12345)
model.fit(Xtrain, ytrain)
predicted = model.predict_proba(Xtest)[:, 1]
ypred = model.predict(Xtest)
result = roc_auc_score(ytest, predicted)
accuracy = model.score(Xtest, ytest)

print('AUC-ROC:', result)
print('accuracy:', accuracy)

AUC-ROC: 0.839349593495935
accuracy: 0.7623318385650224


# Resampling

Искусствено уравняем число индивидов в классах.

Исходная выборка:

In [30]:
gnb_pr = gnb.predict(X_train)
print(classification_report(ytrain, gnb_pr))

print('train accuracy:',gnb.score(X_train, ytrain))
print('test accuracy:',gnb.score(X_test, ytest))

              precision    recall  f1-score   support

           1       0.79      0.63      0.70       341
           2       0.68      0.82      0.75       326

    accuracy                           0.73       667
   macro avg       0.74      0.73      0.72       667
weighted avg       0.74      0.73      0.72       667

train accuracy: 0.7256371814092953
test accuracy: 0.6771300448430493


Случайная наивная избыточная выборка:

In [31]:
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, ytrain)

In [32]:
gnb.fit(X_resampled, y_resampled)

gnb_pr = gnb.predict(X_train)
print(classification_report(ytrain, gnb_pr))

print('train accuracy:',gnb.score(X_train, ytrain))
print('test accuracy:',gnb.score(X_test, ytest))

              precision    recall  f1-score   support

           1       0.80      0.63      0.71       341
           2       0.69      0.84      0.75       326

    accuracy                           0.73       667
   macro avg       0.74      0.74      0.73       667
weighted avg       0.75      0.73      0.73       667

train accuracy: 0.7331334332833583
test accuracy: 0.6636771300448431


Качество классификации стало немного выше, чем при использовании наивного байессовского классфикатора к исходной выборке.

In [33]:
X_resampled, y_resampled = SMOTE().fit_resample(X_train, ytrain)

In [34]:
gnb.fit(X_resampled, y_resampled)

gnb_pr = gnb.predict(X_train)
print(classification_report(ytrain, gnb_pr))

print('train accuracy:',gnb.score(X_train, ytrain))
print('test accuracy:',gnb.score(X_test, ytest))

              precision    recall  f1-score   support

           1       0.79      0.63      0.70       341
           2       0.68      0.82      0.74       326

    accuracy                           0.72       667
   macro avg       0.73      0.72      0.72       667
weighted avg       0.73      0.72      0.72       667

train accuracy: 0.7226386806596702
test accuracy: 0.6816143497757847


Результаты совпадают со случайной избыточной выборкой.

In [35]:
X_resampled, y_resampled = TomekLinks().fit_resample(X_train, ytrain)

In [36]:
gnb.fit(X_resampled, y_resampled)

gnb_pr = gnb.predict(X_train)
print(classification_report(ytrain, gnb_pr))

print('train accuracy:',gnb.score(X_train, ytrain))
print('test accuracy:',gnb.score(X_test, ytest))

              precision    recall  f1-score   support

           1       0.80      0.62      0.70       341
           2       0.68      0.83      0.75       326

    accuracy                           0.72       667
   macro avg       0.74      0.73      0.72       667
weighted avg       0.74      0.72      0.72       667

train accuracy: 0.7241379310344828
test accuracy: 0.672645739910314


Результаты совпадают со случайной избыточной выборкой.

Resampling незначительно улучшил качество классификации во всех трех случаях.

# Случайный лес

По сетке найдем лучшие параметры для случайного леса

In [37]:
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 500, num = 3)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 50, stop = 150, num = 3)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 3)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 10, num = 2)]
bootstrap = [True, False]
GRID = [
    {'estimator': [RandomForestClassifier()],
     'estimator__criterion': ['gini', 'entropy', 'log_loss'],
     'estimator__n_estimators': n_estimators,
     'estimator__max_features': max_features,
     'estimator__max_depth': max_depth,
     'estimator__min_samples_split': min_samples_split,
     'estimator__min_samples_leaf': min_samples_leaf,
     'estimator__bootstrap': bootstrap
     }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', RandomForestClassifier())])

In [38]:
grid_search_RF = GridSearchCV(estimator=PIPELINE, param_grid=GRID, 
                            scoring=make_scorer(accuracy_score), 
                            n_jobs=-1, cv=5, refit=True, verbose=1, 
                            return_train_score=True)

In [39]:
random.seed(1)
grid_search_RF.fit(Xtrain, ytrain)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', None),
                                       ('estimator',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid=[{'estimator': [RandomForestClassifier(criterion='log_loss',
                                                               max_depth=150,
                                                               min_samples_leaf=2,
                                                               n_estimators=500)],
                          'estimator__bootstrap': [True, False],
                          'estimator__criterion': ['gini', 'entropy',
                                                   'log_loss'],
                          'estimator__max_depth': [50, 100, 150],
                          'estimator__max_features': ['log2', 'sqrt'],
                          'estimator__min_samples_leaf': [2, 10],
                          'estimator__min_samples_split': [2, 26, 50],
                          'estimator__n_estimators': [50, 275, 500]}],
             return_train_score=True, scoring=make_scorer(accuracy_score),
             verbose=1)

In [40]:
print(grid_search_RF.score(Xtrain, ytrain))
print(grid_search_RF.best_params_)

1.0
{'estimator': RandomForestClassifier(criterion='log_loss', max_depth=150, min_samples_leaf=2,
                       n_estimators=500), 'estimator__bootstrap': True, 'estimator__criterion': 'log_loss', 'estimator__max_depth': 150, 'estimator__max_features': 'sqrt', 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 500}


In [41]:
print("Test set score: %f" % grid_search_RF.score(Xtest, ytest))

Test set score: 0.828347


Случайный лес показывает лучшие результаты среди всех моделей. Для прогноза для kaggle будем использовать случайный лес.

In [42]:
f = open('class_test.csv','w') 
f.write('Id,sex')
f.write('\n')
grid_search_RF.fit(df.drop(['Sex', 'Id'], axis=1), df[['Sex']].astype(int))
pred = grid_search_RF.predict(df_kag_NA.drop('Id', axis=1))
for i in range(len(df_kag_NA['Id'])):
    f.write(str(df_kag_NA['Id'][i]))
    f.write(',')
    f.write(str(int(pred[i])))
    f.write('\n')

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
